# Simulating APT Programs with MIRaGe
---
In this notebook, we will demonstrate how to use  MIRAGE (Multi-Instrument Ramp Generator) to simulate exposures from Astronomer's Proposal Tool ([APT](https://jwst-docs.stsci.edu/display/JPP/JWST+Astronomers+Proposal+Tool+Overview)) programs using the `mirage.apt` module. This module extracts the instrument, detector, pointing, and filter information from APT program output files, rather than requiring that the user defines them all manually.

The process for APT simulations with MIRaGe is as follows:

- Download needed files from the APT program, and parse those files to access the details and structure of a given program
- Generate catalogs for all exposures in the program
- Generate MIRaGe YAML input files that contain the specifications for each exposure simulation
- Use the `mirage.imaging_simulator` to create seed images and add dark exposures and detector effects (See the [Imaging Simulator example](Imaging_simulator_use_examples.ipynb) for more detail on that process)

### Table of Contents:
1. [Export program information from APT](#export_apt)
2. [Generate catalog files of sources in FOV](#query_cat)
3. [Create YAML files from APT files](#yaml)
5. [Generate the simulated image](#simulate_images)

Appendix A: [Generating data for an entire observation](#simulate_whole_obs)

## Import necessary packages and modules

In [ ]:
# Standard Library Imports
from glob import glob
import os
import shutil
import urllib

# Third Party Imports
import pysiaf
from astropy.io import ascii as asc
from astropy.io import fits
from matplotlib import cm
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt

# Local Imports (from nircam_simulator package)
from mirage import imaging_simulator
from mirage.catalogs import create_catalog
from mirage.utils.utils import ensure_dir_exists
from mirage.yaml import yaml_generator

# View matplotlib plots inline
%matplotlib inline

---
<a id='export_apt'></a>
# Export Program Information from APT

MIRaGe requires APT program output files in order to generate data with unstacked PSFs.

## Get needed files from APT program

For this example, we are using APT output from a parallel NIRCam-FGS commissioning program: program 1071, "NIRCam Absolute FGS-NIRCam Alignment".

In [ ]:
# Define the proposal ID
prop_id = 1071

# Where the pointing and XML file for this particular OTE CAR are
input_dir = os.path.abspath('./apt_data/')
ensure_dir_exists(input_dir)

# Define the names of the pointing and XML files
# NOTE: Change the root if you name your files differently.
root = 'apt_{}'.format(prop_id)
pointing_file = os.path.join(input_dir, '{}.pointing'.format(root))
xml_file = os.path.join(input_dir, '{}.xml'.format(root))

### Option 1: Manually download the `.pointing` and `.xml` files

Open the APT file for the program you want to simulate. If you don't have the file locally, you can load this program in APT by selecting `File > Retrieve from STScI > Retrieve Using Proposal ID` and then entering the program ID (e.g. 1140). (You must be running APT in STScI mode for this retrieval method to be available.)

Export the `.pointing` and `.xml` files for your given proposal in APT by selecting `File > Export...` and selecting both the xml and pointing file options. 

### Option 2: Automatically download the `.pointing` and `.xml` files

If you don't want to bother opening APT and your program is publicly accessible, you can use the below code to download APT files. 

*Note that you must have APT installed for the below command to work.*

In [ ]:
# Build temporary directory
temp_apt_dir = os.path.abspath('./temp_apt/')
if not os.path.isdir(temp_apt_dir):
    os.mkdir(temp_apt_dir)
    print('Create temporary directory to download APT files.')

# Download the APT file
apt_file = os.path.join(temp_apt_dir, '{}.aptx'.format(prop_id))
urllib.request.urlretrieve(
    'http://www.stsci.edu/jwst/phase2-public/{}.aptx'.format(prop_id), 
    apt_file
)
print('Downloaded temporary APT file:', apt_file)

# Determine the user's installation of APT
apt_app = sorted(glob('/Applications/APT*'))[-1].replace(' ', '\ ')
print('Will export pointing and XML files using the following installation of APT:', apt_app)

# Export the APT XML and pointing files
os.system(
    "{}/bin/apt -nogui -export xml,pointing {}".format(apt_app, apt_file)
)

# Move the XML and pointing files to the apt_data directory
os.rename(os.path.join(temp_apt_dir, '{}.xml'.format(prop_id)),
         xml_file)
os.rename(os.path.join(temp_apt_dir, '{}.pointing'.format(prop_id)),
         pointing_file)
print('Downloaded APT pointing file:', pointing_file)
print('Downloaded APT XML file:', xml_file)
    
# Tear down temporary directory
shutil.rmtree(temp_apt_dir)
print('Deleted temporary APT file and directory.')

And before we move on, let's just make sure that worked:

In [ ]:
# Make sure the pointing and XML files exist
assert os.path.exists(pointing_file)
assert os.path.exists(xml_file)

### Define location of output files

The process of generating simulated images with MIRaGe produces a lot of files:
- YAML files carrying the OTE mirror state
- YAML files carrying the specifications for simulations
- FITS files of the simulated seed, dark, and compiled images

Additionally, we must create FITS library files of the segment PSF images in order to simulate images with nonnominal PSFs.

Let's define the directories to save these output files to:

In [ ]:
# Where to save MIRaGe output
out_dir = os.path.join(input_dir, 'output')

---
<a id='query_cat'></a>
# Query online catalogs to generate catalog files of sources in FOV

Next, we need to generate catalog files for the sources around the target in this proposal.

Mirage contains the `create_catalog.for_proposal` function that can be used to create point source and galaxy catalogs from an APT file. This function collects the target RA and Dec values from the proposal, as well as the list of instruments and filters used for the observations. It then runs `mirage.catalog.create_catalog.get_all_catalogs` and `mirage.catalog.create_catalog.galaxy_background` to produce point source and galaxy catalogs. These catalogs can then be used as input when producing the yaml files needed to run Mirage.

In [ ]:
catalog_out = create_catalog.for_proposal(xml_file, pointing_file,
                                          point_source=True,
                                          extragalactic=True,
                                          catalog_splitting_threshold=0.12,
                                          email='someone@somewhere.edu',
                                          out_dir=out_dir,
                                          save_catalogs=True)
ptsrc_cat, gal_cat, ptsrc_names, gal_names, pmap, gmap = catalog_out
cat_dict = {'ptsrc': ptsrc_names[0],
            'gal': gal_names[0]}

## Plot all sources in catalogs

Let's see how many sources we're dealing with here.

In [ ]:
# Plot all queried sources
for catalog_filename in ptsrc_names:
    target_catalog = asc.read(catalog_filename)
    plt.scatter(target_catalog['x_or_RA'], target_catalog['y_or_Dec'], s=1, alpha=.7, label='Point Source Catalog')
for catalog_filename in gal_names:
    target_catalog = asc.read(catalog_filename)
    plt.scatter(target_catalog['x_or_RA'], target_catalog['y_or_Dec'], s=1, alpha=.7, label='Galactic Catalog')
plt.xlabel('Right Ascension [degrees]')
plt.ylabel('Declination [degrees]')
plt.legend()
plt.show()

---
<a id='yaml'></a>
# Create YAML files from APT files

Next, we need to make the YAML files that include all of the parameters for MIRaGe to run.

Use `mirage.yaml.yaml_generator` to make all of the YAML files for the given APT program - one file per exposure.

In [ ]:
# Create a series of data simulator input yaml files from APT files
yaml_dir = os.path.join(out_dir, 'yamls')
yam = yaml_generator.SimInput(input_xml=xml_file, pointing_file=pointing_file,
                              catalogs=cat_dict,
                              verbose=True, output_dir=yaml_dir, simdata_output_dir=out_dir)

# Create all input YAML files (one per each exposure)
yam.create_inputs()

## Choose which yaml (visit/tile) to use

Now that we've generated all of the needed YAML files, we need to choose one to simulate images with. MIRaGE can only generate one simulated exposure at a time, so we need to choose one YAML file in our yamls directory that we will use to produce an image. (See [Appendix A](#simulate_whole_obs) for how use a wrapper to simulate multiple exposures at once with MIRaGe.)

Not every exposure necessarily has the same pointing, so we should choose an exposure that places the target star in the desired detector field-of-view.

### Examine target pointings relative to apertures and V2/V3 references

Looking at the `.pointing` file, let's plot where the target will appear relative to the NIRCam apertures for each unique pointing.

In [ ]:
# Examine apertures and V2/V3 references for each array/subarray
nc_siaf = pysiaf.Siaf('NIRCam')
nc_full = nc_siaf['NRCA1_FULL']

plt.figure(figsize=(15,10))
for apername in sorted(nc_siaf.apernames):
    a = apername
    if ('_FULL' in a) and ('OSS' not in a) and ('MASK' not in a) and (a[-1] != 'P'):
        nc_siaf[a].plot(frame='tel', label=True, fill_color='white')
plt.gca().invert_xaxis()

# Compare V2/V3 of targets (from .pointing file)
all_pointings = set([(v2, v3, filename) for v2, v3, filename in zip(yam.info['v2'], 
                                                                yam.info['v3'], 
                                                                yam.info['yamlfile'])])

print('Example files for each pointing:')
print('--------------------------------')
plotted_points = []
for i_point, (v2, v3, filename) in enumerate(all_pointings):
    if (v2, v3) not in plotted_points:
        plotted_points.append((v2, v3))
        plt.scatter(v2, v3, marker='*', s=500, 
                    label='Pointing {}/{}'.format(i_point + 1, len(all_pointings)))
        print('{}. {}'.format(i_point + 1, filename))

plt.legend()

plt.show()

### Select the YAML to generate an image from

Looking at the pointing figure above, choose one YAML file that we will create a seed image with MIRaGe for. (Be sure to choose a YAML that has a detector and filter that matches the library files you have created so far.)

*See [JDox](https://jwst-docs.stsci.edu/display/JDAT/File+Naming+Conventions+and+Data+Products) for a detailed explanation of the MIRaGe YAML file name format.*

In [ ]:
# Select one YAML to estimate where the sources will be
test_yaml_filename = 'jw01071001001_01103_00001_nrca4.yaml'
test_yaml = os.path.join(yaml_dir, test_yaml_filename)
assert os.path.isfile(test_yaml)
print(test_yaml)

---
<a id='simulate_images'></a>
# Simulate image with MIRaGe

Finally, we can run MIRaGe to generate a seed image simulation of our unstacked mirror state during OTE-01.

From here on out, from the user perspective, the simulation process is identical to that of nominal imaging cases (see the [imaging example notebook](#Imaging_simulator_use_examples.ipynb). To reiterate, it is the specifications made in the YAML files that enable the simulation of unstacked mirror simulations with MIRaGe.

In [ ]:
# Run the image simulator using the input defined in test_yaml
img_sim = imaging_simulator.ImgSim()
img_sim.paramfile = test_yaml
img_sim.create()

In [ ]:
# Plot the seed image, dark image, and final exposure simulation
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(20, 7))
plt.tight_layout()

# Define scale limits and colormap
clim=(0.001, 0.1)
cmap = cm.get_cmap('viridis')
cmap.set_bad(cmap(0))

# Plot seed image
fitsplot = ax1.imshow(img_sim.seedimage, clim=clim, cmap=cmap)
ax1.set_title('Seed Image', size=24)
ax1.invert_xaxis()
ax1.invert_yaxis()

# Plot dark current
ax2.imshow(img_sim.linDark.data[0,-1,:,:] - img_sim.linDark.data[0,0,:,:], clim=clim, cmap=cmap)
ax2.set_title('Dark Current', size=24)
ax2.invert_xaxis()
ax2.invert_yaxis()

# Plot final exposure
file_root = os.path.basename(test_yaml_filename).split('.yaml')[0]
linear_output = os.path.join(out_dir, '{}_linear.fits'.format(file_root))
with fits.open(linear_output) as h:
    lindata = h[1].data
    header = h[0].header
exptime = header['EFFINTTM']
diffdata = (lindata[0,-1,:,:] - lindata[0,0,:,:]) / exptime

ax3.imshow(diffdata, clim=clim, cmap=cmap)
ax3.set_title('Final Exposure Simulation', size=24)
ax3.invert_xaxis()
ax3.invert_yaxis()

# Define the colorbar
cbar_ax = fig.add_axes([1, 0.09, 0.03, 0.87])
cbar = plt.colorbar(fitsplot, cbar_ax)
cbar.set_label('Count Rate', rotation=270, labelpad=30, size=24)

---
<a id='simulate_whole_obs'></a>
# Appendix A: Simulating many exposures at once

Chances are, you don't want to simulate just one exposure from one detector. In order to simulate all of the exposures from a given observation, write a for loop to iterate over all the YAMLs. We include an example for program 1134 observation 1 below.

```python
from mirage import imaging_simulator

# Get all the 1134 Obs 1 NRCA3 yamls
all_yaml_files = glob(os.path.join(yaml_dir, 'jw01134001*.yaml'))
n_yamls = len(all_yaml_files)
print('{} FITS files will be generated.'.format(n_yamls))

# Run imaging_simulator for all YAMLs
for i_yaml, yaml in enumerate(all_yaml_files):
    print('*** SIMULATING YAML {}/{}: {} ***'.format(i_yaml+1, n_yamls, yaml))
    img_sim = imaging_simulator.ImgSim()
    img_sim.paramfile = yaml
    img_sim.create()
```

(If you are impatient and ambitious, you can use Python's `multiprocessing` module to the simulation go faster. Even better on a server with more processors!)